In [1]:
import numpy as np
import matplotlib.pyplot as plt

### Zadanie 1

In [98]:
class LinearReg:
    def __init__(self, eta, c, w1 = 1.0, w2 = 1.0, w0 = 1.0):
        self.eta = eta
        self.c = c
        self.w1 = w1
        self.w2 = w2
        self.w0 = w0
        
    def loss_function(self, X, Y, regularization):
        sum_dif = 0.0

        for inputs, output in zip(X, Y):
            sum_dif += (self.w1 * inputs[0] + self.w2 * inputs[1] + self.w0 - output) ** 2

        if regularization == "L1":
            return sum_dif / (2 * len(X)) + (1 / self.c) * (abs(self.w1) + abs(self.w2) + abs(self.w0))
        elif regularization == "L2":
            return sum_dif / (2 * len(X)) + (1 / self.c) * (self.w1 ** 2 + self.w2 ** 2 + self.w0 ** 2)
        else:
            raise Exception("Podano niepoprawną deskrypcję regularyzacji!")
        
    def update_weights(self, X, Y, regularization):
        d_w1 = 0.0
        d_w2 = 0.0
        d_w0 = 0.0

        for inputs, output in zip(X, Y):
            d_w1 += 2 * self.w1 * inputs[0] ** 2 + 2 * self.w2 * inputs[0] * inputs[1] + 2 * self.w0 * inputs[0] - 2 * inputs[0] * output
            d_w2 += 2 * self.w1 * inputs[0] * inputs[1] + 2 * self.w2 * inputs[1] ** 2 + 2 * self.w0 * inputs[1] - 2 * inputs[1] * output
            d_w0 += 2 * self.w1 * inputs[1] + 2 * self.w2 * inputs[1] + 2 * self.w0 - 2 * output

        d_w1 /= 2 * len(X)
        d_w2 /= 2 * len(X)
        d_w0 /= 2 * len(X)
        
        if regularization == "L1":
            d_w1 += (1 / self.c) * np.sign(self.w1)
            d_w2 += (1 / self.c) * np.sign(self.w2)
            d_w0 += (1 / self.c) * np.sign(self.w0)
        elif regularization == "L2":
            d_w1 += 2 * (1 / self.c) * self.w1
            d_w2 += 2 * (1 / self.c) * self.w2
            d_w0 += 2 * (1 / self.c) * self.w0
        else:
            raise Exception("Podano niepoprawną deskrypcję regularyzacji!")
        
        self.w1 -= self.eta * d_w1
        self.w2 -= self.eta * d_w2
        self.w0 -= self.eta * d_w0
    
    def train(self, X, Y, regularization, iterations):
        for _ in range(iterations):
            self.update_weights(X, Y, regularization)


### Testy:

In [112]:
X = np.random.uniform(-10.0, 10.0, (100, 2))
Y = np.dot(X, [-2.5, 4.2]) + 6.3

In [116]:
model_1 = LinearReg(0.001, 0.1)
model_1.train(X, Y, "L1", 1000)

print(f"""w1: {model_1.w1},\nw2: {model_1.w2},\nw0: {model_1.w0},\nloss_f: {model_1.loss_function(X, Y, "L1")}""")

w1: -2.1772416358132487,
w2: 3.9189326831753215,
w0: -0.0025013258174131554,
loss_f: 83.92109420652402


In [107]:
model_2 = LinearReg(0.001, 0.1)
model_2.train(X, Y, "L2", 1000)

print(f"""w1: {model_2.w1},\nw2: {model_2.w2},\nw0: {model_2.w0},\nloss_f: {model_2.loss_function(X, Y, "L1")}""")

w1: -1.704971586181556,
w2: 2.622957692174877,
w0: 0.47155003890316594,
loss_f: 124.44766622989448


### Zadanie 2

In [119]:
# Generuję dane, x_1, x_2 należą do [-10, 10] x [-10, 10]
X = np.random.uniform(-10.0, 10.0, (500, 2))

# Y = 1.5 * x_1 + 6.5 * x_2 - 4.3 + N(0, 1)
Y = np.dot(X, [1.5, 6.5]) - 4.3 + np.random.normal(0.0, 1.0, 500)

In [120]:
# Podział danych na zbiór walidacyjny i treningowy
x_valid = X[:100]
y_valid = Y[:100]

x_train = X[100:]
y_train = Y[100:]

# Funkcja pomocnicza: trenuje model przy zadanych hiperparametrach oraz wypisuje wartości wag i funkcji kosztu
def train_and_show(X, Y, eta, reg_param):
    model = LinearReg(eta, reg_param)
    model.train(X, Y, "L1", 1500)
    print(f"""eta: {eta}, reg_param: {reg_param}, w1: {model.w1}, w2: {model.w2}, w0: {model.w0}, loss_f: {model.loss_function(X, Y, "L1")}""")

In [121]:
# Wartości eta i reg_param (współczynnik uwzględnienia wartości wag przy liczeniu LF)
etas = [0.05, 0.01, 0.005, 0.001, 0.0005]
reg_params = [10.0, 1.0, 0.1, 0.01, 0.001]

# Iloczyn kartezjański po wartościach eta i reg_param
for eta, reg_param in [(e, r) for e in etas for r in reg_params]:
    train_and_show(x_valid, y_valid, eta, reg_param)

eta: 0.05, reg_param: 10.0, w1: 1.4706397298939866, w2: 6.533692840582917, w0: -3.2080218484029346, loss_f: 1.9414311378608295
eta: 0.05, reg_param: 1.0, w1: 1.4384246056018641, w2: 6.513528664943593, w0: -2.3281392173695616, loss_f: 12.316474791892183
eta: 0.05, reg_param: 0.1, w1: 1.147185738266773, w2: 6.223270871309484, w0: -0.21024396610711865, loss_f: 87.16467873884554
eta: 0.05, reg_param: 0.01, w1: 23.30084149054182, w2: -10.172065763133524, w0: -3.3100514437870956, loss_f: 16878.409875829253
eta: 0.05, reg_param: 0.001, w1: 135.4499013735173, w2: 87.42224472180551, w0: 20.21928377175078, loss_f: 625694.0291842166
eta: 0.01, reg_param: 10.0, w1: 1.470639724759107, w2: 6.533692855572719, w0: -3.2080206373393994, loss_f: 1.941432116796384
eta: 0.01, reg_param: 1.0, w1: 1.4384246020680158, w2: 6.513528675259644, w0: -2.3281383839099843, loss_f: 12.316475450842129
eta: 0.01, reg_param: 0.1, w1: 1.1431765901451314, w2: 6.233928089337262, w0: -0.007087015291142432, loss_f: 85.9583854

In [122]:
# To samo co powyżej tylko na zbiorze treningowych i dla wybranych wartości eta i reg_param
train_and_show(x_train, y_train, 0.005, 10.0)

eta: 0.005, reg_param: 10.0, w1: 1.4942176337555253, w2: 6.5101110842854295, w0: -3.4213648209061494, loss_f: 1.9801657569577185


### Zadanie 3

In [84]:
# Wagi modelu z zadania
w1 = -0.39682866
w2 = -0.82497163
w0 = 0.11932619

# Klasyfikowany punkt
x1 = 3.0
x2 = 3.0

# P że klasa = 1
p_1 = 1 / (1 + np.exp(-(w1 * x1 + w2 * x2 + w0)))

# P że klasa = 0
p_0 = 1 - p_1

# Wyszło tak jak w zadaniu
print(p_0, p_1)

0.9719706839711546 0.02802931602884544


### Zadanie 4

In [87]:
# klasa 0:
w0 = np.array([0.01577681, -0.05318977, 0.21160694, 0.14655186, -0.24476592, -0.00318551, -0.20914917, -0.26505038, -0.02765106, 0.26778537])
w00 = 0.0294703

# klasa 1:
w1 = np.array([0.20327639, 0.0990516, -0.39851296, -0.1045002, 0.02344135, -0.06340171, 0.19494493, 0.54495946, -0.18417782, -0.2844254])
w10 = -0.26754233

# klasa 2:
w2 = np.array([-0.2190532, -0.04586183, 0.18690602, -0.04205165, 0.22132456, 0.06658722, 0.01420424, -0.27990908, 0.21182888, 0.01664003])
w20 = 0.23807203

# obserwacje:
row = np.array([1.89149379, -0.39847585, 1.63856893, 0.01647165, 1.51892395, -3.52651223, 1.80998823, 0.58810926, -0.02542177, -0.52835426])

# obliczenia:
p_0_raw = np.exp(np.sum(w0 * row) + w00)
p_1_raw = np.exp(np.sum(w1 * row) + w10)
p_2_raw = np.exp(np.sum(w2 * row) + w20)

p_0 = p_0_raw / (p_0_raw + p_1_raw + p_2_raw)
p_1 = p_1_raw / (p_0_raw + p_1_raw + p_2_raw)
p_2 = p_2_raw / (p_0_raw + p_1_raw + p_2_raw)

# Wyszło tak jak w zadaniu
print(p_0, p_1, p_2)

0.16470455808850257 0.5029713823568449 0.33232405955465244


### Zadanie 5

Zwiększając wartość c zmniejszamy wartość współczynnika a (c = 1 / a), co w konsekwencji przekłada się na mniejszą karę od wielkości wag.
Na wykresie widać rozgałęzienie bo od pewnego c bardziej opłaca się (w kontekście wartości funkcji kosztu) dopasowywać wagi modelu tak aby ten 
model lepiej odwzorowywał dane (zmniejszać błąd) niż dbać o to aby wagi były małe (bliskie zera).